In [63]:
import geopandas as gpd

from CENSAr.aggregation import named_aggregation
from CENSAr.datasources import (
    personas_radios_prov,
    tipoviv_radios_prov,
    radios_prov,
    radios_eph_censo_2010,
    persproy_depto_2025,
)
from CENSAr.modeling_tools import (
    simulate_total_var, 
    simulate_cat_var
)
from CENSAr.contexto_urbano import *
from CENSAr.utils import *
import warnings
warnings.filterwarnings("ignore")

In [66]:
# GEOMETRIAS
envolvente = radios_eph_censo_2010(aglo_idx="08")
chaco_2001 = radios_prov(year=2001, prov="chaco", mask=envolvente)
chaco_2010 = radios_prov(year=2010, prov="chaco", mask=envolvente)

# TABLAS REDATAM - HOGARES/VIVIENDAS
tipo_2001 = tipoviv_radios_prov(
    year=2001,
    prov="chaco",
    var_types={"link": "object"},
)
tipo_2001_geo = chaco_2001.set_index("link").join(tipo_2001.set_index("link"))
tipo_2010 = tipoviv_radios_prov(
    year=2010,
    prov="chaco",
    var_types={"link": "object"},
)
tipo_2010_geo = chaco_2010.set_index("link").join(tipo_2010.set_index("link"))

[11:05:26] INFO     https://storage.googleapis.com/python_mdg/censar_carto/radios_eph_json.zip   ]8;id=11425;file://D:\JOB\CEEU\CENSAr\CENSAr\datasources.py\datasources.py]8;;\:]8;id=989469;file://D:\JOB\CEEU\CENSAr\CENSAr\datasources.py#103\103]8;;\

[11:05:39] INFO     loading                                                                      ]8;id=817855;file://D:\JOB\CEEU\CENSAr\CENSAr\datasources.py\datasources.py]8;;\:]8;id=899121;file://D:\JOB\CEEU\CENSAr\CENSAr\datasources.py#112\112]8;;\
                    `https://storage.googleapis.com/python_mdg/censar_data/tipo_vivienda_radios_                   
                    chaco_2001.csv`                                                                                

           INFO     loading                                                                      ]8;id=522067;file://D:\JOB\CEEU\CENSAr\CENSAr\datasources.py\datasources.py]8;;\:]8;id=739434;file://D:\JOB\CEEU\CENSAr\CENSAr\datasources.py#112\112]8;;\
                    `https://storage.googleapis.com/python_mdg/censar_data/tipo_vivienda_radios_                   
                    chaco_2010.csv`                                                                                

In [13]:
tipo_vivienda_agg_2001 = named_aggregation(
    tipo_2001_geo, name="tipo vivienda particular"
)
tipo_vivienda_agg_2010 = named_aggregation(
    tipo_2010_geo, name="tipo vivienda particular"
) 

[15:24:19] INFO     Applying named aggregation `tipo vivienda particular`                                ]8;id=807179;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py\core.py]8;;\:]8;id=429001;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py#67\67]8;;\

           WARNING  Extra columns found for mapping situacion de calle: {'persona viviendo en la        ]8;id=44957;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py\core.py]8;;\:]8;id=916456;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py#123\123]8;;\
                    calle'}. Extra columns will be ignored.                                                        

           INFO     Applying named aggregation `tipo vivienda particular`                                ]8;id=778667;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py\core.py]8;;\:]8;id=500437;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py#67\67]8;;\

           WARNING  Extra columns found for mapping formal: {'casa b', 'casa a'}. Extra columns will be ]8;id=904904;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py\core.py]8;;\:]8;id=913794;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py#123\123]8;;\
                    ignored.                                                                                       

           WARNING  Extra columns found for mapping situacion de calle: {'persona viviendo en la        ]8;id=677755;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py\core.py]8;;\:]8;id=740625;file://D:\JOB\CEEU\CENSAr\CENSAr\aggregation\core.py#123\123]8;;\
                    calle'}. Extra columns will be ignored.                                                        

In [19]:
informal_simulated_distribution = simulate_cat_var(
    gdf_var_01=tipo_vivienda_agg_2001,
    gdf_var_10=tipo_vivienda_agg_2010,
    base_year="0110",
    forecast_year="2020",
    forecast_gdf=tipo_vivienda_agg_2010,
    pct_val=1.5,
    catname={"2001": "informal", "2010": "informal"},
    tot_colname="total"
)

tipo_vivienda_agg_2010["cat_sim_2020"] = tipo_vivienda_agg_2010.index.map(informal_simulated_distribution)
tipo_vivienda_agg_2010["informal_2020"] = tipo_vivienda_agg_2010["informal"] + tipo_vivienda_agg_2010["cat_sim_2020"]

# Asignar polígonos

In [235]:
zona_inf = tipo_vivienda_agg_2010.copy()
zona_inf.reset_index(inplace=True)
zona_inf.head()

,link,geometry,total,formal,informal,situacion de calle,cat_sim_2020,informal_2020
0,221402105,"POLYGON ((-58.94747 -27.51435, -58.94728 -27.5...",385,364,21,0,12.0,33.0
1,221402106,"POLYGON ((-58.94180 -27.51231, -58.94176 -27.5...",491,473,18,0,16.0,34.0
2,221402102,"POLYGON ((-58.94171 -27.51378, -58.94176 -27.5...",262,261,1,0,6.0,7.0
3,221402807,"POLYGON ((-58.95469 -27.50759, -58.95327 -27.5...",137,136,1,0,3.0,4.0
4,221402101,"POLYGON ((-58.93704 -27.50790, -58.93715 -27.5...",303,303,0,0,0.0,0.0


In [136]:
zona_sup = gpd.read_file("../Data_generada/zonas_resistencia.geojson")
# zona_sup = gpd.read_file("../Data_generada/barrios.shp")
zona_sup.rename(columns={'id':'barrio'})
campo_superior = 'grupo' # barrio o grupo
zona_sup.head()

,id,grupo,geometry
0,12,a,"MULTIPOLYGON (((-59.02214 -27.44504, -59.02214..."
1,2,b,"MULTIPOLYGON (((-59.07446 -27.46655, -59.07446..."
2,1,c,"MULTIPOLYGON (((-58.97279 -27.47111, -58.97279..."
3,20,d,"MULTIPOLYGON (((-59.04312 -27.39748, -59.04311..."
4,27,e,"MULTIPOLYGON (((-59.06636 -27.41843, -59.06635..."


In [137]:
zona_inf['area_tot'] = zona_inf.area
zonas_inter = gpd.overlay(zona_sup, zona_inf, how='intersection')

zonas_inter['area_par'] = zonas_inter.area
zonas_inter['area_porc'] = round(zonas_inter['area_par'] / zonas_inter['area_tot'], 3)
zonas_inter = zonas_inter.sort_values(by='area_porc', ascending=False).reset_index(drop=True)
zonas_inter.drop_duplicates(subset='link', keep='first', inplace=True)

zonas_inter.drop(columns=['area_par', 'area_tot', 'area_porc'], inplace=True)
zonas_inter = zona_inf.merge(zonas_inter[['link', 'barrio', 'grupo']], on='link')
zonas_inter.head(2)

436
537
430
430


,link,geometry,total,formal,informal,situacion de calle,cat_sim_2020,informal_2020,area_tot,id,grupo
0,221402105,"POLYGON ((-58.94747 -27.51435, -58.94728 -27.5...",385,364,21,0,12.0,33.0,0.000030,1,c
1,221402106,"POLYGON ((-58.94180 -27.51231, -58.94176 -27.5...",491,473,18,0,16.0,34.0,0.000153,1,c


In [138]:
area_sup_tot = zonas_inter.groupby(campo_superior)[['total', 'formal', 'informal']].sum().reset_index()
area_sup_tot.rename(columns={'total':'sup_total', 'formal':'sup_formal', 'informal':'sup_informal'}, inplace=True)
zonas_inter = zonas_inter.merge(area_sup_tot, on=campo_superior)
zonas_inter.head()

,link,geometry,total,formal,informal,situacion de calle,cat_sim_2020,informal_2020,area_tot,id,grupo,sup_total,sup_formal,sup_informal
0,221402105,"POLYGON ((-58.94747 -27.51435, -58.94728 -27.5...",385,364,21,0,12.0,33.0,0.000030,1,c,18150,17474,675
1,221402106,"POLYGON ((-58.94180 -27.51231, -58.94176 -27.5...",491,473,18,0,16.0,34.0,0.000153,1,c,18150,17474,675
2,221402102,"POLYGON ((-58.94171 -27.51378, -58.94176 -27.5...",262,261,1,0,6.0,7.0,0.000010,1,c,18150,17474,675
3,221402101,"POLYGON ((-58.93704 -27.50790, -58.93715 -27.5...",303,303,0,0,0.0,0.0,0.000017,1,c,18150,17474,675
4,221402109,"MULTIPOLYGON (((-58.95328 -27.51955, -58.95317...",44,22,22,0,0.0,22.0,0.000007,1,c,18150,17474,675


In [139]:
lista_disimilitud_indice = list()
for g in list(zonas_inter[campo_superior].unique()):
    zonas_inter_grupo = zonas_inter.loc[zonas_inter[campo_superior] == g]
    grupo_formal = zonas_inter_grupo['formal'] / zonas_inter_grupo['sup_formal']
    grupo_informal = zonas_inter_grupo['informal'] / zonas_inter_grupo['sup_informal']
    
    disimilitud_indice = round((abs(grupo_formal - grupo_informal).sum()) * 0.5, 3)
    lista_disimilitud_indice.append([g, disimilitud_indice])
    print(g, "==>", disimilitud_indice)

c ==> 0.468
b ==> 0.691
a ==> 0.544
d ==> 0.543
e ==> 0.538


In [140]:
zona_sup_indce = zona_sup.merge(pd.DataFrame(lista_disimilitud_indice, columns=[campo_superior, 'indice']), on=campo_superior)
zona_sup_indce.to_file("../Data_generada/zonas_resistencia_"+campo_superior+".geojson")

# Por buffer con los vecinos

In [246]:
zona_inf['indice'] = -1
zona_inf['indice_count'] = 0
zona_inf['area_tot'] = zona_inf.area

for b in [[0.003, "3"], [0.005, '5'], [0.007, "7"],  [0.009, '9']]:
    for r in range(len(zona_inf)):
        zona_inf_test = zona_inf.loc[[r]]
        zona_inf_buffer = zona_inf_test[['geometry']].copy()
        zona_inf_buffer.geometry = zona_inf_test.buffer(b[0])

        zonas_inter_buffer = gpd.overlay(zona_inf_buffer, zona_inf, how='intersection')
        zonas_inter_buffer['area_par'] = zonas_inter_buffer.area
        zonas_inter_buffer['area_porc'] = round(zonas_inter_buffer['area_par'] / zonas_inter_buffer['area_tot'], 3)
        zonas_inter_buffer = zonas_inter_buffer.loc[zonas_inter_buffer.area_porc > 0.30]

        if len(zonas_inter_buffer) > 1:
            zonas_inter_buffer['formal_tot'] = zonas_inter_buffer['formal'].sum()
            zonas_inter_buffer['informal_tot'] = zonas_inter_buffer['informal'].sum()

            lista_valores = abs((zonas_inter_buffer['formal'] / zonas_inter_buffer['formal_tot']) - (zonas_inter_buffer['informal'] / zonas_inter_buffer['informal_tot']))
            disimilitud_indice = round((lista_valores.sum()) * 0.5, 3)

            zona_inf.loc[r, 'indice'] = disimilitud_indice
            zona_inf.loc[r, 'indice_count'] = len(zonas_inter_buffer)

        zonas_inter_buffer_2 = zonas_inter_buffer.loc[zonas_inter_buffer.link != list(zona_inf.loc[r, ['link']])[0]]
        zonas_inter_buffer_2['formal_tot'] = zonas_inter_buffer_2['formal'].sum()
        zonas_inter_buffer_2['informal_tot'] = zonas_inter_buffer_2['informal'].sum()
        lista_valores_2 = abs((zonas_inter_buffer_2['formal'] / zonas_inter_buffer_2['formal_tot']) - (zonas_inter_buffer_2['informal'] / zonas_inter_buffer_2['informal_tot']))
        disimilitud_indice_2 = round((lista_valores_2.sum()) * 0.5, 3)

        zona_inf.loc[r, 'indice_vecindad'] = disimilitud_indice_2

        # detección de radios que al agregarlos empeoran la similitud del entorno
        zona_inf['dif_indices_entornos'] = zona_inf['indice_vecindad'] - zona_inf['indice'] 
        zona_inf['dif_indices_entornos_abs'] = abs(zona_inf['indice_vecindad'] - zona_inf['indice'] )

    zona_inf.loc[zona_inf.indice != -1].to_file("../Data_generada/indice_radios_"+b[1]+"30.geojson")